## 1. Libraries and settings

In [ ]:
import numpy as np
import random
import pandas as pd 
from pylab import mpl, plt
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

import math, time
import itertools
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable

import vectorbtpro as vbt

from datetime import date

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import warnings
warnings.simplefilter("ignore", UserWarning)


# #hparams

# # Update these dimensions based on your dataset

hidden_dim = 32
num_layers = 2

num_epochs = 5
learning_rate=0.01
step_size=30
gamma=0.9

dropout_rate=0.2


In [ ]:
df = pd.read_csv('2ySOLdata1h.csv')
pd.set_option('future.no_silent_downcasting', True)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.set_index('timestamp', inplace=True)

df['signal'] = df['signal'].replace({'SignalNone': 1, 'SignalLong': 2, 'SignalShort': 0})
df.ffill()


In [ ]:
data = vbt.Data.from_data(df)
features = data.run("talib", mavp=vbt.run_arg_dict(periods=14))
data.data['symbol'] = pd.concat([data.data['symbol'], features], axis=1)
data.data['symbol'].drop(['Open', 'High', 'Low'], axis=1, inplace=True)
# This will drop columns from the DataFrame where all values are NaN
data.data['symbol'] = data.data['symbol'].dropna(axis=1, how='all')


In [ ]:

open_price = data.get('Open')
high_price = data.get('High')
low_price = data.get('Low')
close_price = data.get('Close')

data.data['symbol'] = data.data['symbol'].dropna()


In [ ]:
# Define the tuple column names and the 'signal' column
columns_to_keep = data.data['symbol'].drop('signal', axis=1).columns.tolist()

# Filter the DataFrame to keep only the specified columns
filtered_df = data.data['symbol'][columns_to_keep]
# Print the current column names to verify their format



In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Assuming filtered_df is your DataFrame and the last column is 'signal' or any categorical label

# Select all columns except the last one for scaling
columns_to_scale = filtered_df.columns[:-1]

# Initialize the MinMaxScaler
scaler = MinMaxScaler()
# Fit and transform the data for the columns to scale
# .values.reshape(-1, 1) is used to transform the data into the correct shape for scaling
# Note: .fit_transform expects a 2D array, hence the reshaping is necessary
scaled_columns.columns = scaled_columns.columns.astype(str)
scaled_columns = scaler.fit_transform(filtered_df[columns_to_scale])

# Create a DataFrame from the scaled columns
scaled_df = pd.DataFrame(scaled_columns, index=filtered_df.index, columns=columns_to_scale)

# Add the unscaled 'signal' column back to the scaled DataFrame
scaled_df['signal'] = filtered_df['signal']



In [ ]:
# function to create train, test data given stock data and sequence length
def load_data(stock, look_back):
    data_raw = stock.values # convert to numpy array
    data = []
    
    # create all possible sequences of length look_back
    for index in range(len(data_raw) - look_back): 
        data.append(data_raw[index: index + look_back])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:-1]
    y_train = data[:train_set_size,-1,-1]
    
    x_test = data[train_set_size:,:-1,:-1]
    y_test = data[train_set_size:,-1,-1]
    
    return [x_train, y_train, x_test, y_test]

look_back = 60 # choose sequence length
x_train, y_train, x_test, y_test = load_data(filtered_df, look_back)
print('x_train.shape = ',x_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_train.shape = ',y_train.shape)
print('y_test.shape = ',y_test.shape)

In [ ]:
x_train_numeric = x_train.astype(np.float32)
x_test_numeric = x_test.astype(np.float32)
y_train_numeric = y_train.astype(np.float32)
y_test_numeric = y_test.astype(np.float32)

In [ ]:
# Check for MPS (GPU on M1 Mac) availability and set it as the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Assuming y_train is your target labels tensor for the training data
# and it's already in the form of a 1D tensor of class indices (0 to C-1)

# Convert y_train to a numpy array if it's a tensor
if isinstance(y_train, torch.Tensor):
    y_train_np = y_train.cpu().numpy()
else:
    y_train_np = y_train  # Assuming y_train is already a numpy array

# Calculate class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_np), y=y_train_np)

# Convert class weights to a tensor
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

# Move class weights to the same device as your model and data
class_weights_tensor = class_weights_tensor.to(device)  # device could be 'cpu' or 'cuda'

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert your numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(x_train_numeric, dtype=torch.float).to(device)
y_train_tensor = torch.tensor(y_train_numeric, dtype=torch.long).to(device)  # Use torch.long for classification labels

X_test_tensor = torch.tensor(x_test_numeric, dtype=torch.float).to(device)
y_test_tensor = torch.tensor(y_test_numeric, dtype=torch.long).to(device)
# Create TensorDatasets
# train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
# # Create DataLoaders
# batch_size = 20  # You can adjust the batch size
# train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)  # Typically no need to shuffle the test set

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


input_dim = X_train_tensor.shape[2]  # Number of features
output_dim = 3  # Number of classes

# Here we define our model as a class
class BiLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout_rate):
        super(BiLSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # Bidirectional LSTM Layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)
        
        # Fully connected layer
        # The input dimension is twice the hidden_dim because it's bidirectional
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(x.device)  # times 2 for bidirectional
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(x.device)  # times 2 for bidirectional
        
        # Forward propagate LSTM
        out, (hn, cn) = self.lstm(x, (h0, c0))
        
        # Apply dropout to the output of the LSTM
        out = self.dropout(out)
        
        # Concatenate the hidden states from both directions
        out = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1)
        
        # Pass the concatenated hidden states to the fully connected layer
        out = self.fc(out)
        
        return out

# Create the model with bidirectional LSTM
model = BiLSTMClassifier(input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers, output_dim=output_dim, dropout_rate=dropout_rate)
model.to(device)  # Move your model to the MPS device

# Loss function, optimizer, and scheduler remain the same
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimiser, step_size=step_size, gamma=gamma)



In [ ]:
num_epochs = 30
print_epochs = 1

# Create the model with bidirectional LSTM
# model = BiLSTMClassifier(input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers, output_dim=output_dim, dropout_rate=dropout_rate)
model.to(device)  # Move your model to the MPS device


# hist will track the loss for now
hist = np.zeros(num_epochs)

# Ensure your model is in training mode
model.train()

for epoch in range(num_epochs):
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    #model.hidden = model.init_hidden()
    
    # Forward pass
    y_train_pred = model(X_train_tensor)

    loss = loss_fn(y_train_pred, y_train_tensor.long())

    hist[epoch] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()
    
    scheduler.step()
 
    
    if epoch % print_epochs == 0 and epoch !=0:  # Adjust logging frequency according to your preference
        print(f"Epoch {epoch}, Loss: {loss.item()}")
        
        # Optional: Add accuracy calculation or other metrics here
        # Note: You'd typically calculate validation metrics here using a separate validation set
        



In [ ]:
# # Adjust the figure size
# plt.figure(figsize=(6, 3))

# # Plot the training loss
# plt.plot(hist, label="Training loss")
# plt.legend()

# # Show the plot
# plt.show()

In [ ]:
model.eval()


In [ ]:
X_test_tensor_gpu = X_test_tensor.to(device)


In [ ]:
with torch.no_grad():
    y_test_pred = model(X_test_tensor_gpu)
    
    # Convert logits to probabilities for each class
    probabilities = torch.softmax(y_test_pred, dim=2)  # Assuming the model outputs logits with shape [batch_size, sequence_length, num_classes]

    # Get the predicted class labels for each time step
    _, predicted_labels = torch.max(probabilities, dim=2)

    # Move the predictions back to CPU if needed, and convert to numpy for further processing or evaluation
    predicted_labels_numpy = predicted_labels.cpu().numpy()

In [ ]:
predicted_labels_flat = predicted_labels_numpy.flatten()
predicted_labels_flat.shape

In [ ]:
df_split = data.data['symbol'][-len(predicted_labels_numpy):].copy()
df_split.loc[:, "signal"] = predicted_labels_numpy


# signal = df_split['signal']
# entries = signal == 2
# exits = signal == 0

In [ ]:
pf = vbt.Portfolio.from_signals(
    close=df_split.Close, 
    long_entries=entries, 
    long_exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)

In [ ]:
vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 600
vbt.settings['plotting']['layout']['height'] = 300

In [ ]:
pf.plot({"orders", "cum_returns"}, settings=dict(bm_returns=False)).show()

In [ ]:
pf.stats()

In [ ]:
stats = pf.stats()
total_return = stats['Total Return [%]']
orders = stats['Total Orders']
print("Total Orders:", orders)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Convert tensors to numpy arrays for use with Scikit-Learn
true_labels = y_test.cpu().numpy()
pred_labels = predicted_labels.cpu().numpy()

precision = precision_score(true_labels, pred_labels, average='macro')  # 'macro' for unweighted mean
recall = recall_score(true_labels, pred_labels, average='macro')
f1 = f1_score(true_labels, pred_labels, average='macro')
conf_matrix = confusion_matrix(true_labels, pred_labels)

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print('Confusion Matrix:\n', conf_matrix)